In [5]:
import astropy.io.fits as fits
import numpy as np
import glob
import os
from scipy.ndimage import gaussian_filter
import pandas as pd
import shutil
from function import *

In [ ]:
make1Expdark

## dark特定の事前準備

処理手順  
ダークの名前を読み込むためのリスト作成   
ダークを特定して、データフレームに必要な情報を入れる  
そのcsvを該当の年のフォルダに移す  

In [ ]:
def searchdarkfile(year):  #first
    basefolder = r'/mnt/e/DataFrame/'
    allframe = pd.read_excel(basefolder + "COM_"+year+".xlsx", skiprows=0)
    allframe.dropna(how="all", axis=0)
    darkframe = allframe[(allframe["DATA_TYP"].str.contains("DARK")) |
                        (allframe["OBJECT2"].str.contains("DARK"))]
    darkframe = darkframe.reset_index(drop=True)
    darkframe.to_csv(r'/mnt/e/DataFrame/dark/darkframe_' +
                    year + r'.csv', encoding="utf-8")
    return darkframe

COMAのファイルから、ダークのヘッダーを読み込み必要な情報を取得する。  
データフレームに格納する  
また、dark画像をdarkフォルダを作成して移動する。  

### メモ
INS_VER 2001-09-29以降のものにしか、YSTRTがなかった  
YSTRTの指定により、y軸のピクセル数が変わってくる  
1なら240pixだが、40なら160pixというように

In [ ]:
def makedarkcsv(year):
    basefolder = r'/mnt/e/'+year + r'/'
    darkdf = pd.DataFrame(
        columns=["FRAME_ID", "DATE", "PIXTIM", "RRSTRT", "YSTRT", "CoAdd", "Q_CHEB"])
    darkframe = pd.read_csv(r'/mnt/e/DataFrame/dark/darkframe_' +
                            year + r'.csv')
    for darkfile in darkframe["#FRAME_ID"]:
        # header読み込み
        darkfilepath = basefolder + darkfile + ".fits"
        darkheader = readheader(darkfilepath)

        list1 = [darkheader["FRAMEID"], darkheader["DATE-OBS"], darkheader["Q_PIXTIM"],
                darkheader["Q_RRSTRT"], darkheader["Q_YSTRT"], darkheader["Q_CHAM"], darkheader["Q_CHEB"]]
        add_row = pd.DataFrame([list1], columns=darkdf.columns)
        darkdf = pd.concat([darkdf, add_row], ignore_index=True)
        # dark画像を移動
        os.makedirs(basefolder+r'dark', exist_ok=True)
        shutil.move(darkfilepath, basefolder+r'dark/')
    darkdf.to_csv(basefolder+r'dark/dark.csv', index=False, encoding="utf-8")


In [ ]:
makedarkcsv("2003")

## 天体画像の入力に対してdarkを特定(dfを返す)

In [2]:
def identify(obsfilename):
    obsheader = readheader(obsfilename)
    #年を取得
    year = obsheader["DATE-OBS"][:4]
    #DARK.csvにアクセス
    darkcsv = r'/mnt/e/'+year +r'/dark/dark.csv'
    darkdf = pd.read_csv(darkcsv)
    #一致条件を指定する
    Q_PIXTIM = obsheader["Q_PIXTIM"]
    Q_RRSTRT = obsheader["Q_RRSTRT"]
    DATE = obsheader["DATE-OBS"]
    Q_YSTRT = obsheader["Q_YSTRT"]
    A = (darkdf["PIXTIM"] == Q_PIXTIM)
    B = (darkdf["RRSTRT"] == Q_RRSTRT)
    C = (darkdf["DATE"] == DATE)
    D = (darkdf["YSTRT"] == Q_YSTRT)
    df = darkdf[A & B & C & D].reset_index(drop=True)
    return df, year

## 特定後、1Expあたりの平均画像作成

In [ ]:
def make1Expdark(obsfile):
    df = identify(obsfile)[0]
    year = identify(obsfile)[1]
    darkbasefolder = r'/mnt/e/' + year +r'/dark/'
    obsdata= readdata(obsfile)
    #obsdataと同じshapeの0のnumpyを作成
    darkdata = np.zeros(list(obsdata.shape))
    if len(df) == 0:
        print("該当のダークが存在しない")
        #ここにはその場合の処理を記載する。
        
    #特定したダークファイル一枚一枚に対して、Q_CHEBが同じものを抽出??

    # df[df["Q_CHEB"].duplicated()]
    for darkfile in df["FRAME_ID"]:
        darkfilepath = darkbasefolder + darkfile +r'.fits'
        darkdata += np.mean(readdata(darkfilepath),axis=1,keepdims=True)
    #meanをとる
    meandark = darkdata / len(df)
    #1Expあたりに直す
    meandark_1 = meandark / int(df["Q_CHEB"][0])
    return meandark_1

In [6]:
make1Expdark

<function function.make1Expdark(obsfile)>

検証用  
使用するのは、2003年の11/12にとられたCOMA00042689.fitsである。HD1255

In [9]:
obsf = r'/mnt/e/2003/COMA00042689.fits'

In [10]:
make1Expdark(obsf)

KeyError: 0

In [ ]:
darkfile = identify(obsfile)[0]["FRAME_ID"][0]
darkfile

In [ ]:
darkbasefolder = r'/mnt/e/' + "2003" + r'/dark/'
darkfile = identify(obsfile)[0]["FRAME_ID"][0]
darkfilepath =darkbasefolder + darkfile + r'.fits'
d2 = np.mean(readdata(darkfilepath),axis=1,keepdims=True)

In [ ]:
obsdata = readdata(obsfile)

In [ ]:
empty_array = np.zeros(list(obsdata.shape))
d3 = empty_array + d2